# Στοιχεία ομάδας
# Ομάδα Β3
# Μπάκος Παναγιώτης ( 03113014 )
# Φώτος Νικόλαος ( 03113122 )

---

In [2]:
!pip install deap
from deap import base, creator
from deap import tools
import random
import time
from math import *
from deap import algorithms
import numpy as np

# Άσκηση 3: Βελτιστοποίηση συναρτήσεων με Γενετικούς Αλγόριθμους
v1.1, 16/02/2018

<img src="http://infinity77.net/global_optimization/_images/Alpine01.png" alt="Alpine01" style="width: 500px;"/>

Στόχος της άσκησης είναι η βελτιστοποίηση συναρτήσεων χωρίς παραγώγους (derivative free optimization) με χρήση Γενετικών Αλγόριθμων μέσω της βιβλιοθήκης DEAP. Η βελτιστοποίηση χωρίς παραγώγους είναι ιδιαίτερα χρήσιμη σε περιπτώσεις όπου η αντικειμενική συνάρτηση $f$ δεν είναι διαθέσιμη (black-box optimization) ή σε περίπτωσεις που είναι, είναι δύσκολο ή μη πρακτικό να υπολογιστούν οι παράγωγοί της. Για παράδειγμα η  $f$ μπορεί να μην είναι διαφορίσιμη, η παραγώγιση της μπορεί να είναι δύσκολη ή να απαιτεί πολύ χρόνο,  η $f$ να περιέχει θόρυβο έτσι ώστε οι μέθοδοι που βασίζονται σε απειροστικές διαφορές να μην είναι αποτελεσματικές ή να υπάρχουν πολλά τοπικά ελάχιστα μεταξύ άλλων. 

Για περισσότερα πάνω στη βελτιστοποίηση χωρίς παραγώγους και τις εφαρμογές της, μπορείτε να συμβουλευτείτε το ["Gradient Free Optimization"](http://adl.stanford.edu/aa222/lecture_notes_files/chapter6_gradfree.pdf).

---

## Συναρτήσεις προς βελτιστοποίηση
Στην ομάδα μας ανατέθηκαν οι εξής συναρτήσεις :
* η μη-κλιμακούμενη συνάρτηση (non-scalable, δηλαδή ορίζεται για συγκεκριμένες διαστάσεις του πεδίου ορισμού) συνάρτηση **Keane** (69)
* η κλιμακούμενη (scalable, μπορεί να οριστεί για οποιαδήποτε διάσταση D) συνάρτηση **Stepint** (141)

Όλες οι συναρτήσεις είναι προς **ελαχιστοποίηση**.

Θα χρησιμοποιήσουμε ως γονίδια πραγματικούς αριθμούς και ως χρωμοσώματα λίστες γονιδίων μήκους ίσο με τη διάσταση του πεδίου ορισμού της συνάρτησης.

---

## Επιλογή κατάλληλου Γενετικού Αλγόριθμου

Ο κάθε Γενετικός Αλγόριθμος χαρακτηρίζεται α) από τους γενετικούς τελεστές του δηλαδή τον ορισμό της διασταύρωσης, μετάλλαξης και επιλογής και β) από τη στρατηγική εξέλιξης που καθορίζει την εξέλιξη του πληθυσμού μέσω της διαδικασίας επιλογής, καθώς και των  πιθανοτήτων διασταύρωσης και μετάλλαξης.

Όπως πολύ συχνά ισχύει σε προβλήματα Μηχανικής Μάθησης, η επιλογή του κατάλληλου αλγόριθμου εξαρτάται από το κάθε πρόβλημα και τα δεδομένα του. Συνεπώς για κάθε συνάρτηση θα πρέπει να δοκιμαστούν διαφορετικοί συνδυασμοί τελεστών και στρατηγικών.

### Αξιολόγηση Γενετικών Αλγόριθμων

Μπορούμε να πούμε ότι ένας ΓΑ είναι πιθανόν να έχει καλύτερα χαρακτηριστικά για ένα δεδομένο πρόβλημα αν:
- βρίσκει τη λύση (βέλτιστη τιμή) με μεγαλύτερη συχνότητα
- βρίσκει τη βέλτιστη λύση με μικρότερο αριθμό αποτιμήσεων της συνάρτησης καταλληλότητας

Πρακτικά, για να αξιολογήσουμε τους αλγόριθμους της άσκησης θα φτιάξουμε μια συνάρτηση που θα αναλύει τα logfiles ως εξής:  

Για ένα μέγιστο αριθμό γύρων (MAX_ROUNDS)  
&nbsp;&nbsp;&nbsp;&nbsp;Για ένα μέγιστο αριθμό γενεών (MAX_GENS)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Για κάθε ΓΑ με δεδομένους γενετικούς τελεστές και στρατηγική εξέλιξης υπολογίζουμε τα εξής μεγέθη:

 
- **avg.min:** ο μέσος όρος της ελάχιστης τιμής που πέτυχε για όλους τους γύρους.
- **avg.evals:** o μέσος όρος των συνολικών αποτιμήσεων (άθροιση όλων των γενεών) που έκανε για όλους τους γύρους.
- **avg.time:** ο μέσος χρόνος εκτέλεσης των γύρων. 

Επιπρόσθετα, επειδή δεν είναι εφικτό για κάθε συνάρτηση να αναμένουμε ότι σε λογικά χρονικά διαστήματα (ή και γενικά) θα βρούμε την ακριβή βέλτιστη λύση, ορίζουμε ως επιτυχία την ποσότητα *GOAL + DELTA*, όπου GOAL το ελάχιστο της συνάρτησης (δίνεται στο pdf) και DELTA μια θετική ποσότητα που θα ορίζουμε εμείς κάθε φορά (διαφορετική για κάθε συνάρτηση και διαφορετική ανάλογα του πόσο κοντά στη λύση θέλουμε να φτάσουμε). Με βάση την ποσότητα GOAL + DELTA ορίζουμε επιπλέον τα εξής μεγέθη:
- **successes:** αριθμός φορών που o ΓΑ βρήκε τιμή μικρότερη του GOAL + DELTA. Προφανώς το βέλτιστο είναι MAX_ROUNDS.
- **success avg. gen:** o μέσος όρος των γενεών που χρειάστηκε ο αλγόριθμος για να πέσει κάτω από GOAL + DELTA, για τους γύρους με επιτυχία. Ουσιαστικά μας ενδιαφέρει η πρώτη γενιά που πετυχαίνει τιμή κάτω από GOAL + DELTA. Σε περίπτωση που δεν πετυχαίνει αυτό το στόχο καμία γενιά, θέτουμε ‘None’ σε αυτή την ποσότητα.
- **success avg. min:** ο μέσος όρος των ελαχίστων των πρώτων επιτυχημένων γενεών για τους γύρους με επιτυχία. Παρόμοια, θέτουμε ‘None’ σε περίπτωση αποτυχίας.
- **success avg. evals:** ο μέσος όρος αποτιμήσεων που χρειάστηκαν για να πάρουμε την πρώτη τιμή κάτω από GOAL + DELTA. Και πάλι  θέτουμε ‘None’ σε περίπτωση αποτυχίας.

Φτιάχνουμε τέλος μια βοηθητική συνάρτηση που τυπώνει συγκεντρωτικά τα αποτελέσματα ως εξής:

| operators                      | strategy                      | successes | s.avg.min | s.avg.evals | s.avg.gens | avg.evals | avg.min   | avg.time |
|--------------------------------|-------------------------------|-----------|-----------|-------------|------------|-----------|-----------|----------|
| cxBlendLow,muGauLow,selTourLow | eaSimple 200 0.8 0.2          | 1         | 8.508e-42 | 16487       | 96         | 17014     | 9.115e-34 | 1.898    |
| cxBlendLow,muGauLow,selTourLow | eaMuPlusLambda 50 150 0.8 0.2 | 1         | 3.885e-41 | 14750       | 98         | 15050     | 1.631e-21 | 1.591    


---

## 1. Βελτιστοποίηση μη κλιμακούμενης συνάρτησης

Ο τύπος της μη-κλιμακούμενης συνάρτησης που μας ανατέθηκε είναι ο εξής :

\begin{equation}
    f(x_1,x_2) = - \frac{\sin^2{(x_1-x_2)}\sin^2{(x_1+x_2)}}{\sqrt{x_1^2+x_2^2}} , 0 \leq x_1,x_2 \leq 10
\end{equation}
---

## Βασικές έννοιες: Creator, Base, Fitness

Οι δύο βασικές έννοιες του DEAP είναι οι Creator και Base. 

O **Creator** είναι ένα μέτα - εργοστάσιο δημιουργίας κλάσεων που θα χρησιμεύσουν στον γενετικό αλγόριθμο. Η **Base** είναι ένα δομοστοιχείο που παρέχει δύο βασικές δομές (κλάσεις) για την κατασκευή του γενετικού: το **Toolbox**, που θα χρησιμοποιήσουμε για να αποθηκέυσουμε (εισάγουμε) τους τελεστές και την (εικονική) κλάση **Fitness** που θα χρησιμοποιήσουμε για να κατασκευάσουμε το μέλος καταλληλότητας του κάθε ατόμου.

Ας πούμε ότι θέλουμε να ορίσουμε μια συνάρτηση καταλληλότητας προς *ελαχιστοποίηση*:

Η συνάρτηση `create` λαμβάνει τουλάχιστον __δύο ορίσματα__: το όνομα της κλάσης που θα δημιουργήσουμε και τη βασική κλάση από την οποία θα κληρονομίσει. Τα ακόλουθα ορίσματα αποτελούν χαρακτηριστικά της κλάσης. Στην περίπτωση του παραδείγματος, η νέα κλάση `FitnessMin` κληρονομεί την `base.Fitness` με χαρακτηριστικό την πλειάδα weights (-1.0,). Το (-1.0,) σημαίνει ότι θέλουμε να ελαχιστοποιήσουμε ένα μόνο κριτήριο (την τιμή της συνάρτησης f(x<sub>1</sub>,x<sub>2</sub>) ).

Στη συνέχεια ορίζουμε την κλάση του **ατόμου** το οποίο κληρονομεί από τον τύπο *list* και περιλαμβάνει το χαρακτηρηστικό `FitnessMin`:

In [0]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

Μπορούμε τώρα να ορίσουμε ένα στιγμιότυπο ατόμου και να υπολογίσουμε/ορίσουμε την καταλληλότητα του:

In [5]:
ind = creator.Individual([-3.02,4.59])
#ind.fitness.values = (sum(ind),)

print(ind)
print type(ind)
print type(ind.fitness)
#print ind.fitness.values

SyntaxError: ignored

---

### Εύρεση βέλτιστου συνδυασμού τελεστών - στρατηγικής


* Ως συνάρτηση καταλληλότητας "evaluate" εγγράφουμε την **`evalKeaneFunc`** που εμείς ορίσαμε προηγουμένως. 

* Στη συνέχεια εγγράφουμε ως "mate" τον τελεστή διασταύρωσης . Επιλέγουμε να δοκιμάσουμε τους τελεστές [**`cxUniform`**](http://deap.readthedocs.io/en/master/api/tools.html#deap.tools.cxUniform)(με τιμές για την indpb 0.05, 0.4) και [**`cxBlend`**](http://deap.readthedocs.io/en/master/api/tools.html#deap.tools.cxBlend)(με τιμές για την alpha 0.1, 0.5). 

* Στη συνέχεια ορίζουμε τον τελεστή μετάλλαξης "mutate".  Επιλέγουμε να δοκιμάσουμε τους τελεστές [**`mutGaussian`**](http://deap.readthedocs.io/en/master/api/tools.html#deap.tools.mutGaussian)(με τιμές για την indpb 0.01, 0.3) και [**`mutShuffleIndexes`**](http://deap.readthedocs.io/en/master/api/tools.html#deap.tools.mutShuffleIndexes)(με τιμές για την indpb 0.01, 0.3). 

* Τέλος επιλέγουμε εγγράφουμε τον τελεστή επιλογής "select" που χρησιμοποιεί την [**`selTournament`**](http://deap.readthedocs.io/en/master/api/tools.html#deap.tools.selTournament) . Η υπερπαράμετρος που θα δοκιμάζουμε εδώ είναι το μέγεθος διοργάνωσης. Δοκιμάζουμε τις τιμές 4 και 12, κάτι που σημαίνει ότι θα διαλέξουμε 4 και 12 αντίστοιχα τυχαία άτομα του πληθυσμού, τα συγκρίνουμε και κρατάμε το καλύτερο (ένας ακόμη τρόπος να υλοποιήσουμε το survival of the fittest).

Έχουμε δηλαδή συνολικά 32 συνδυασμούς τελεστών για να αξιολογήσουμε.

Για κάθε συνδυασμό τελεστών θα εξετάσουμε τρεις στρατηγικές: τον [**απλό ΓΑ**](http://deap.readthedocs.io/en/master/api/algo.html#deap.algorithms.eaSimple) και τις στρατηγικές εξέλιξης [**“μ+λ”**](http://deap.readthedocs.io/en/master/api/algo.html#deap.algorithms.eaMuPlusLambda) και [**“μ,λ”**](http://deap.readthedocs.io/en/master/api/algo.html#deap.algorithms.eaMuCommaLambda).

Αρχικά θα προσδιορίσουμε τους καταλληλότερους συνδυασμούς τελεστών-στρατηγικών. Για το λόγο αυτό θα θέσουμε ένα σχετικά μικρό αριθμό γύρων (π.χ. 5-10) και μέγιστων γενεών (π.χ. 50-100-150) και σταθερές τιμές στις παραμέτρους πληθυσμού και τις πιθανότητες διασταύρωσης και μετάλλαξης.

---

Πρώτα θα ορίσουμε όλες τις απαραίτητες συναρτήσεις. 

Αρχικά ορίζουμε τη συνάρτηση καταλληλότητας, την ίδια τη συνάρτηση Keane δηλαδή.

Ο βασικός τρόπος για να επιβάλουμε περιορισμούς όσον αφορά το πεδίο ορισμού της συνάρτησης είναι να επιβάλουμε μια ποινή στην τιμή της καταλληλότητας στα άτομα που είναι εκτός των ορίων που έχουμε θέσει. Ορίζουμε δύο συναρτήσεις, τη "feasible" που μας επιστρέφει True αν όλα τα $x_i$ είναι εντός του διαστήματος και False αλλιώς και την "distance" που μας ποσοτικοποιεί πόσο εκτός ορίων είναι ένα άτομο. Συγκεκριμένα επιλέγουμε η απόσταση να είναι το απόλυτο άθροισμα σε όλες τις διαστάσεις της απόστασης από το όριο. Θα μπορούσαμε να κάνουμε και άλλες επιλογές όπως πχ να χρησιμοποιήσουμε μια τετραγωνική συνάρτηση της απόστασης.



In [0]:
def evalKeaneFunc(ind):
    x1 = ind[0];
    x2 = ind[1];
    val = -pow(sin(x1-x2),2) * pow(sin(x1+x2),2) / sqrt(x1*x1+x2*x2)
    return (val,)

MIN_BOUND = np.array([0]*2) #όσες και οι διαστάσεις της συνάρτησης
MAX_BOUND = np.array([10]*2)

def feasible( indiv ):
    if any( indiv < MIN_BOUND) or any( indiv > MAX_BOUND):
        return False
    return True

def distance( indiv ) :
    dist = 0.0
    for i in range (len( indiv )) :
        penalty = 0
        if ( indiv [i] < MIN_BOUND[i]) : penalty = 0 - indiv [i]
        if ( indiv [i] > MAX_BOUND[i]) : penalty = indiv [i] - 10
        dist = dist + penalty
    return dist

Θα ορίσουμε τώρα μία συνάρτηση που θα ελέγχει την αποτελεσματικότητα ενός συνδυασμού.

In [0]:
def one_combo_test(c_var,m_var,s_var,c_flag,m_flag,flag,MAX_GENS,POPULATION,delta_penalty,target,CXPB,MUTPB):
    VERBOSE = False #print log

    toolbox = base.Toolbox()
    toolbox.register("attr_real", random.uniform, 0, 10) #range των x1 & x2 variables
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_real, 2)  #όσες και οι διαστάσεις της συνάρτησης
    toolbox.register("population", tools.initRepeat, list, toolbox.individual) 

    toolbox.register( "evaluate", evalKeaneFunc)
    toolbox.decorate( "evaluate", tools.DeltaPenality (feasible, delta_penalty, distance))

    pop = toolbox.population(n=POPULATION)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    if (c_flag==0):
        toolbox.register("mate", tools.cxUniform, indpb = c_var)
    else:
        toolbox.register("mate", tools.cxBlend, alpha = c_var)

    if(m_flag==0):
        toolbox.register("mutate", tools.mutGaussian, mu = 0, sigma=1.0, indpb=m_var)
    else:
        toolbox.register("mutate", tools.mutShuffleIndexes, indpb=m_var)


    toolbox.register("select", tools.selTournament, tournsize=s_var)

    t_start = time.time()
    if(flag==0): 
        pop_new, logbook = algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=MAX_GENS, stats=stats, halloffame=hof, verbose=VERBOSE)
    elif(flag==1): 
        pop_new, logbook =  algorithms.eaMuPlusLambda(pop, toolbox, mu=POPULATION/5, lambda_=4*POPULATION/5, cxpb=CXPB, mutpb=MUTPB, ngen=MAX_GENS, stats=stats, halloffame=hof, verbose=VERBOSE)
    elif(flag==2):
        pop_new, logbook = algorithms.eaMuCommaLambda(pop, toolbox, mu=POPULATION/5, lambda_=4*POPULATION/5, cxpb=CXPB, mutpb=MUTPB, ngen=MAX_GENS, stats=stats, halloffame=hof, verbose=VERBOSE)
    t_finish = time.time()

    success = 0
    if ( evalKeaneFunc(hof[0])[0] <=target ):
        success = 1 #οταν ο ΓΑ βρίσκει τιμή μικρότερη του GOAL+DELTA

    curr_tot_evals =  0
    curr_min_first_gen = 0
    curr_min_first_val = 0
    for j in range(MAX_GENS-1, -1, -1):
        curr_tot_evals += logbook[j]['nevals']
        # θα κρατήσουμε την μικρότερη γενιά για την οποία ισχύει ότι ο ΓΑ πέτυχε τιμή μικρότερη από GOAL+DELTA, για γύρο που στέφθηκε με επιτυχία
        if( logbook[j]['min'] <= target ):
            curr_min_first_gen = j+1
            curr_min_first_val = logbook[j]['min']
    curr_min_evals = 0
    for j in range(0,curr_min_first_gen,1):
        curr_min_evals += logbook[j]['nevals']

    return evalKeaneFunc(hof[0])[0], curr_tot_evals, t_finish-t_start,success,curr_min_first_gen, curr_min_first_val, curr_min_evals

Θα ορίσουμε τώρα μία συνάρτηση που θα ελέγξει την αποτελεσματικότητα των 32 συνδυασμών που αναφέρθηκαν προηγουμένως.

Οι συγκεκριμένοι 32 συνδυασμοί είναι οι εξής : 

|A/A|crossover hyperparameter|mutation hyperparameter|selection hyperparameter|
|---|------------------------|-----------------------|------------------------|
|1  |cxUniformLow            |muGauLow               |selTourLow              |
|2  |cxUniformLow            |muGauLow               |selTourHigh             |
|3  |cxUniformLow            |muGauHigh              |selTourLow              |
|4  |cxUniformLow            |muGauHigh              |selTourHigh             |
|5  |cxUniformHigh           |muGauLow               |selTourLow              |
|6  |cxUniformHigh           |muGauLow               |selTourHigh             |
|7  |cxUniformHigh           |muGauHigh              |selTourLow              |
|8  |cxUniformHigh           |muGauHigh              |selTourHigh             |
|9  |cxBlendLow              |muGauLow               |selTourLow              |
|10 |cxBlendLow              |muGauLow               |selTourHigh             |
|11 |cxBlendLow              |muGauHigh              |selTourLow              |
|12 |cxBlendLow              |muGauHigh              |selTourHigh             |
|13 |cxBlendHigh             |muGauLow               |selTourLow              |
|14 |cxBlendHigh             |muGauLow               |selTourHigh             |
|15 |cxBlendHigh             |muGauHigh              |selTourLow              |
|16 |cxBlendHigh             |muGauHigh              |selTourHigh             |
|17 |cxUniformLow            |muShuIndLow            |selTourLow              |
|18 |cxUniformLow            |muShuIndLow            |selTourHigh             |
|19 |cxUniformLow            |muShuIndHigh           |selTourLow              |
|20 |cxUniformLow            |muShuIndHigh           |selTourHigh             |
|21 |cxUniformHigh           |muShuIndLow            |selTourLow              |
|22 |cxUniformHigh           |muShuIndLow            |selTourHigh             |
|23 |cxUniformHigh           |muShuIndHigh           |selTourLow              |
|24 |cxUniformHigh           |muShuIndHigh           |selTourHigh             |
|25 |cxBlendLow              |muShuIndLow            |selTourLow              |
|26 |cxBlendLow              |muShuIndLow            |selTourHigh             |
|27 |cxBlendLow              |muShuIndHigh           |selTourLow              |
|28 |cxBlendLow              |muShuIndHigh           |selTourHigh             |
|29 |cxBlendHigh             |muShuIndLow            |selTourLow              |
|30 |cxBlendHigh             |muShuIndLow            |selTourHigh             |
|31 |cxBlendHigh             |muShuIndHigh           |selTourLow              |
|32 |cxBlendHigh             |muShuIndHigh           |selTourHigh             |
Αργότερα θα αναφερθούμε σε αυτούς με τους Α/Α τους.

In [0]:
def combination_testing(un_list,bl_list,ga_list,sh_list,se_list,target,flag,MAX_GENS,POPULATION,CXPB,MUTPB,rnd,mute):
    delta_penalty = 5.0
    # crossover hyperparameters (low,high)
    cxUniform_indpb_list = un_list
    cxBlend_alpha_list = bl_list
    # mutation hyperparameters (low,high)
    mutGaussian_indpb_list = ga_list
    mutShuffleIndexes_indpb_list = sh_list
    #selection hyperparameters (low,high)
    selTournament_tour_list = se_list

    ## Μετρικές απόδοσης
    min_val = []        # ελάχιστη τιμή που πέτυχε ο ΓΑ στο συγκεκριμένο γύρο
    tot_evals = []     # συνολικές αποτιμήσεις(άθροιση όλων των γενεών) του συγκεκριμένου γύρου
    time_of_round = [] # χρόνος εκτέλεσης τou γύρou. 

    #Επιπρόσθετα, επειδή δεν είναι εφικτό για κάθε συνάρτηση να αναμένουμε ότι σε λογικά χρονικά διαστήματα (ή και γενικά)
    #θα βρούμε την ακριβή βέλτιστη λύση, ορίζουμε ως επιτυχία την ποσότητα *GOAL + DELTA*, όπου GOAL το ελάχιστο της συνάρτησης
    #(δίνεται στο pdf) και DELTA μια θετική ποσότητα που θα ορίζουμε εμείς κάθε φορά (διαφορετική για κάθε συνάρτηση και 
    #διαφορετική ανάλογα του πόσο κοντά στη λύση θέλουμε να φτάσουμε). Με βάση την ποσότητα GOAL + DELTA ορίζουμε επιπλέον τα εξής μεγέθη:
    success = []       # 1 αν ο ΓΑ βρήκε τιμή μικρότερη του GOAL+DELTA.
    min_first_gen = [] # γενιά στην οποία ο ΓΑ πέτυχε να πέσει κάτω από GOAL+DELTA
    min_first_val = [] # τιμή που πέτυχε ο ΓΑ στην πρώτη γενιά που κατάφερε να πέσει κάτω απο GOAL+DELTA
    min_evals = []     # συνολικές αποτιμήσεις που χρειάστηκαν για να πάρουμε την πρώτη τιμή κάτω από GOAL + DELTA

    round_start = time.time()
    ##############################################################################################################################################################################################
    ## CROSSOVER : UNIFORM, MUTATION : GAUSSIAN
    for curr1 in cxUniform_indpb_list:
        for curr2 in mutGaussian_indpb_list:
            for curr3 in selTournament_tour_list:
                f1,f2,f3,f4,f5,f6,f7 = one_combo_test(curr1,curr2,curr3,0,0,flag,MAX_GENS,POPULATION,delta_penalty,target,CXPB,MUTPB)
                min_val.append(f1)  ;  tot_evals.append(f2)   ;  time_of_round.append(f3)  ;  success.append(f4)  ;  min_first_gen.append(f5)  ;  min_first_val.append(f6)  ;  min_evals.append(f7)                
    ###############################################################################################################################################################################################
    ## CROSSOVER : BLEND, MUTATION : GAUSSIAN
    for curr1 in cxBlend_alpha_list:
        for curr2 in mutGaussian_indpb_list:
            for curr3 in selTournament_tour_list:
                f1,f2,f3,f4,f5,f6,f7 = one_combo_test(curr1,curr2,curr3,1,0,flag,MAX_GENS,POPULATION,delta_penalty,target,CXPB,MUTPB)
                min_val.append(f1)  ;  tot_evals.append(f2)   ;  time_of_round.append(f3)  ;  success.append(f4)  ;  min_first_gen.append(f5)  ;  min_first_val.append(f6)  ;  min_evals.append(f7)                
    ###############################################################################################################################################################################################
    ## CROSSOVER : UNIFORM, MUTATION : SHUFFLE_INDEXES
    for curr1 in cxUniform_indpb_list:
        for curr2 in mutShuffleIndexes_indpb_list:
            for curr3 in selTournament_tour_list:
                f1,f2,f3,f4,f5,f6,f7 = one_combo_test(curr1,curr2,curr3,0,1,flag,MAX_GENS,POPULATION,delta_penalty,target,CXPB,MUTPB)
                min_val.append(f1)  ;  tot_evals.append(f2)   ;  time_of_round.append(f3)  ;  success.append(f4)  ;  min_first_gen.append(f5)  ;  min_first_val.append(f6)  ;  min_evals.append(f7)                
    ###############################################################################################################################################################################################       
    ## CROSSOVER : BLEND, MUTATION : SHUFFLE_INDEXES
    for curr1 in cxBlend_alpha_list:
        for curr2 in mutShuffleIndexes_indpb_list:
            for curr3 in selTournament_tour_list:
                f1,f2,f3,f4,f5,f6,f7 = one_combo_test(curr1,curr2,curr3,1,1,flag,MAX_GENS,POPULATION,delta_penalty,target,CXPB,MUTPB)
                min_val.append(f1)  ;  tot_evals.append(f2)   ;  time_of_round.append(f3)  ;  success.append(f4)  ;  min_first_gen.append(f5)  ;  min_first_val.append(f6)  ;  min_evals.append(f7)                  
    ###############################################################################################################################################################################################
    round_finish = time.time()

    if (mute==False):
        print('Round {:<3} finished. Elapsed time: {:<6}'.format(rnd+1,str(round(round_finish-round_start,6))))
    return min_val,tot_evals,time_of_round,success,min_first_gen,min_first_val,min_evals

Στη συνέχεια ορίζουμε μία συνάρτηση που ελέγχει για κάθε στρατηγική την απόδοση των πιο πάνω συνδυασμών, τρέχοντας την πιο πάνω συνάρτηση `MAX_ROUNDS` φορές. Θα κρατήσουμε μέσους όρους των μετρικών που επιστρέφονται από τη συνάρτηση `combination_testing`

In [0]:
def strategy_round_testing(flag,goal,delta,MAX_ROUNDS,MAX_GENS,POPULATION,CXPB,MUTPB,un_list,bl_list,ga_list,sh_list,se_list,mute):
    tot_combos = len(un_list)*len(ga_list)*len(se_list) + len(un_list)*len(sh_list)*len(se_list) + len(bl_list)*len(ga_list)*len(se_list) + len(bl_list)*len(sh_list)*len(se_list)
    ev_strategies = ['SimpleEA','Mu+Lambda','Mu,Lambda']
    target = goal+delta # ψάχνουμε τη βέλτιστη με έναν βαθμό ανοχής λύση

    #cummulative arrays
    sum_min_val = [0]*tot_combos
    sum_tot_evals = [0]*tot_combos
    sum_time_of_round = [0]*tot_combos
    sum_success = [0]*tot_combos # αριθμός φορών που o ΓΑ βρήκε τιμή μικρότερη του GOAL + DELTA. Προφανώς το βέλτιστο είναι MAX_ROUNDS.
    sum_min_first_gen = [0]*tot_combos
    sum_min_first_val = [0]*tot_combos
    sum_min_evals = [0]*tot_combos

    for k in range(0,MAX_ROUNDS,1):
        min_val,tot_evals,time_of_round,success,min_first_gen,min_first_val,min_evals =  combination_testing(un_list,bl_list,ga_list,sh_list,se_list,target,flag,MAX_GENS,POPULATION,CXPB,MUTPB,k,mute)

        sum_min_val = [x+y for x,y in zip(min_val,sum_min_val)]
        sum_tot_evals = [x+y for x,y in zip(tot_evals,sum_tot_evals)]
        sum_time_of_round = [x+y for x,y in zip(time_of_round,sum_time_of_round)]
        
        sum_success = [x+y for x,y in zip(success,sum_success)]
        sum_min_first_gen = [x+y for x,y in zip(min_first_gen,sum_min_first_gen)]
        sum_min_first_val = [x+y for x,y in zip(min_first_val,sum_min_first_val)]
        sum_min_evals = [x+y for x,y in zip(min_evals,sum_min_evals)]


    avg_min_val =       [i/MAX_ROUNDS for i in sum_min_val]       # ο μέσος όρος της ελάχιστης τιμής που πέτυχε για όλους τους γύρους.
    avg_tot_evals =     [i/MAX_ROUNDS for i in sum_tot_evals]     # o μέσος όρος των συνολικών αποτιμήσεων (άθροιση όλων των γενεών) που έκανε για όλους τους γύρους.
    avg_time_of_round = [i/MAX_ROUNDS for i in sum_time_of_round] # ο μέσος χρόνος εκτέλεσης των γύρων.

    avg_min_first_gen = [0]*tot_combos # o μέσος όρος των γενεών που χρειάστηκε ο αλγόριθμος για να πέσει κάτω από GOAL + DELTA, για τους γύρους με επιτυχία. 
                               # Ουσιαστικά μας ενδιαφέρει η πρώτη γενιά που πετυχαίνει τιμή κάτω από GOAL + DELTA. Σε περίπτωση που δεν πετυχαίνει 
                               # αυτό το στόχο καμία γενιά, θέτουμε ‘None’ σε αυτή την ποσότητα.
    avg_min_first_val = [0]*tot_combos # ο μέσος όρος των ελαχίστων των πρώτων επιτυχημένων γενεών για τους γύρους με επιτυχία. Θέτουμε ‘None’ σε περίπτωση αποτυχίας.
    avg_min_evals = [0]*tot_combos     # ο μέσος όρος αποτιμήσεων που χρειάστηκαν για να πάρουμε την πρώτη τιμή κάτω από GOAL + DELTA. Θέτουμε ‘None’ σε περίπτωση αποτυχίας.

    for i in range(0,tot_combos,1): #αν sum_success[i]==0, τότε ποτέ δεν είχαμε επιτυχία για το συγκεκριμένο συνδυασμό
        if (sum_min_first_gen[i]==0):
            avg_min_first_gen[i] = None
            avg_min_first_val[i] = None
            avg_min_evals[i] = None

        else:
            avg_min_first_gen[i] = sum_min_first_gen[i]/sum_success[i]
            avg_min_first_val[i] = sum_min_first_val[i]/sum_success[i]
            avg_min_evals[i]     = sum_min_evals[i]/sum_success[i]
    print('-'*103)
    print '\n\nPresenting Statistics for Evolution Strategy={:<9} |  MAX_ROUNDS={:<3} | MAX_GENS={:<3} | POPULATION={:<4} | delta={:<6} | CXPB={:<3} | MUTPB={:<6}'.format(ev_strategies[flag],MAX_ROUNDS,MAX_GENS,POPULATION,delta,CXPB,MUTPB)
    print '\n\n{:<3}| {:<9} | {:<15} | {:<11} | {:<10} | {:<9} | {:<16} | {:<8}'.format('Comb.', 'Successes', 's.avg.min','s.avg.evals','s.avg.gens','avg.evals','avg.min','avg.time')
    print '-'*103
    for i in range(0,tot_combos,1):
        print '{:<4} | {:<9} | {:<15} | {:<11} | {:<10} | {:<9} | {:<16} | {:<8}'.format(i+1, sum_success[i], avg_min_first_val[i],avg_min_evals[i],avg_min_first_gen[i],avg_tot_evals[i],avg_min_val[i],round(avg_time_of_round[i],5))
    print '-'*103,'\n\n'


Ώρα να δοκιμάσουμε διάφορες τιμές στις παραμέτρους `MAX_ROUNDS`, `MAX_GENS`, `POPULATION`, `delta`, και για διαφορετικές τιμές του `flag`(διαφορετικές στρατηγικές εξέλιξης)

1) Αρχικά δοκιμάσαμε για 10 γύρους, 30 γενιές, και πληθυσμό 30. Η τιμή του delta είχε οριστεί στο 0.01. Όπως είναι φυσικό, πήραμε αρκετά successes, αλλά με κακή προσέγγιση του ελαχίστου. Πολλοί συνδυασμοί εμφάνισαν επιτυχία, αλλά οι περισσότεροι σε πολύ χαμηλό ποσοστό όσον αφορά το πλήθος των γύρων που τρέξαμε (~10-20%)
    
2) Αυξήσαμε τις γενιές στις 45. Παρατηρήσαμε κάποιοιυς συνδυασμούς να μην εμφανίζουν επιτυχία, αναμενόμενο. Αυξάνοντας κι άλλο τις γενιές στις 60 συνεχίζεται το ίδιο μοτίβο. Πάντα παρατηρούμε και μια μικρή αύξηση στο χρόνο. Εμφανίζονται σιγά σιγά συνδυασμοί που εμφανίζουν επιτυχία για όλους τους γύρους που τους δοκιμάσαμε.
    
3) Πλέον με τις 60 γενιές, ώρα να κάνουμε το delta 10 φορές μικρότερο. Ο χρόνος όπως είναι αναμενόμενο παραμένει ίδιος. Αλλά τα αποτελέσματα όσον αφορά την επιτυχία πέφτουν, καθώς θεωρούμε επιτυχή μία πρόβλεψη πιο σπάνια (σε μικρότερο εύρος αριθμών γύρω από το ελάχιστο) . Δεν έχουμε συνδυασμό με 100% επιτυχία, αλλά τα αποτελέσματα που παίρνουμε είναι όλο και πιο κοντά στο ελάχιστο.

4) Δοκιμάζουμε να αυξήσουμε τις γενιές στις 75. Αύξηση στο χρόνο, αλλά και καλύτερα αποτελέσματα. Με delta ίσο με 0.0005 και 100 γενιές εμφανίζεται max 70% επιτυχία στους εξεταζόμενους συνδυασμούς. Επιλέγουμε όμως τελικά τα παρακάτω combinations, καθώς θα βελτιωθούν παραπάνω για μεγαλύτερο πληθυσμό, γενιές και γύρους εκτέλεσης (και άλλες υπερπαραμέτρους που θα εξετάσουμε αργότερα).


Όσον αφορά τις διαφορετικές στρατηγικές εξέλιξης, η καλύτερη είναι ο απλός εξελικτικός αλγόριθμος `SimpleEA`. Οι υπόλοιποι δεν καταφέρνουν να εμφανίσουν μεγάλη επιτυχία.

In [0]:
flag = 0 # simpleEA ###flag = 1 ### M+L #flag = 2 # M,L
MAX_ROUNDS = 10
MAX_GENS = 75
POPULATION = 40
goal = -0.673667521146855 # http://benchmarkfcns.xyz/benchmarkfcns/kealefcn.html
delta = 0.0005
CXPB = 0.7
MUTPB = 0.3
mute = False

un_list=[0.05, 0.35]
bl_list=[0.15 , 0.5]
ga_list=[0.01, 0.3]
sh_list=[0.01, 0.3]
se_list=[3,10]

strategy_round_testing(flag,goal,delta,MAX_ROUNDS,MAX_GENS,POPULATION,CXPB,MUTPB,un_list,bl_list,ga_list,sh_list,se_list,mute)

In [0]:
flag=1 #M+L
strategy_round_testing(flag,goal,delta,MAX_ROUNDS,MAX_GENS,POPULATION,CXPB,MUTPB,un_list,bl_list,ga_list,sh_list,se_list,mute)
flag=2 #M,L
strategy_round_testing(flag,goal,delta,MAX_ROUNDS,MAX_GENS,POPULATION,CXPB,MUTPB,un_list,bl_list,ga_list,sh_list,se_list,mute)

### Τελική βελτιστοποίηση

#### Βελτιστοποίηση πιθανοτήτων διασταύρωσης και μετάλλαξης

Παρατηρούμε από τα παραπάνω ότι ο καλύτερος συνδυασμός είναι ο **16**. Στο συγκεκριμένο χρησιμοποιούμε τον τελεστή `Blend` για διασταύρωση(alpha=0.5), τον `Gaussian` για μετάλλαξη(indpb=0.3) και την τιμή 10 για τον Tournament Selector. Καλά αποτελέσματα εμφανίζει επίσης ο συνδυασμός **15**, o οποίος απλά έχει μικρότερο tournament size, τον οποίο δεν προτιμούμε όμως λόγω του μικρού tournament size.

Για το συνδυασμό αυτό τελεστών-στρατηγικής και σταθερό αριθμό γύρων, πληθυσμού και γενεών, κάνουμε grid search στους τελεστές διασταύρωσης και μετάλλαξης στο διάστημα [0.1 0.9] (να αθροίζουν στη μονάδα) για να βρούμε το βέλτιστο συνδυασμό τους.

In [0]:
flag = 0 # simpleEA ###flag = 1 ### M+L #flag = 2 # M,L
MAX_ROUNDS = 25
MAX_GENS = 75
POPULATION = 50
goal = -0.673667521146855 # http://benchmarkfcns.xyz/benchmarkfcns/kealefcn.html
delta = 0.0001
mute = True

# κρατάμε μόνο τον συνδυασμό που μας ενδιαφέρει
un_list=[]
bl_list=[0.5]
ga_list=[0.3]
sh_list=[]
se_list=[10]

CXPB_arr = np.arange(0.1,0.95,0.05)
for curr_cxpb in CXPB_arr:
    curr_mutpb = 1-curr_cxpb
    strategy_round_testing(flag,goal,delta,MAX_ROUNDS,MAX_GENS,POPULATION,curr_cxpb,curr_mutpb,un_list,bl_list,ga_list,sh_list,se_list,mute)

Οι μεταβλητές `CXPB` και `MUTPB` συμβολίζουν τα εξής :
- CXPB  : ποσοστό κληρονομικότητας από γονείς
- MUTPB : ποσοστό τυχαίας μετάλλαξης

Παρατηρούμε πως το καλύτερο αποτέλεσμα το παίρνουμε για τις τιμές του `CXPB` στο διάστημα 0.45 έως 0.55, με τις αντίστοιχες τιμές για το `MUTPB`, ώστε πάντα `CXPB+MUTPB=1`

Έχοντας λοιπόν καταλήξει και σε κατάλληλα `CXPB=0.45` και `MUTPB=0.55`, μπορούμε να τρέξουμε τον αλγόριθμό μας με βέλτιστες παραμέτρους και να ελέγξουμε το αποτέλεσμα.

#### Εύρεση βέλτιστης τιμής

Κάνουμε ένα run του βέλτιστου αλγόριθμου που προέκυψε στα προηγούμενα βήματα με ένα μεγάλο αριθμό γενεών και πληθυσμού για να πάρετε μια βέλτιστη τιμή για τη συνάρτηση (σε λογικά χρονικά πλαίσια). Τυπώνουμε τη βέλτιστη τιμή, το συνολικό αριθμό αποτιμήσεων και το χρόνο εκτέλεσης.

In [0]:
flag = 0 # simpleEA ###flag = 1 ### M+L #flag = 2 # M,L
MAX_ROUNDS = 30
MAX_GENS = 200
POPULATION = 150
goal = -0.673667521146855 # http://benchmarkfcns.xyz/benchmarkfcns/kealefcn.html
delta = 0.000001
CXPB = 0.45
MUTPB = 0.55
mute = False

# κρατάμε μόνο τον συνδυασμό που μας ενδιαφέρει
un_list=[]
bl_list=[0.5]
ga_list=[0.3]
sh_list=[]
se_list=[10]

strategy_round_testing(flag,goal,delta,MAX_ROUNDS,MAX_GENS,POPULATION,CXPB,MUTPB,un_list,bl_list,ga_list,sh_list,se_list,mute)

In [0]:
flag = 0 # simpleEA ###flag = 1 ### M+L #flag = 2 # M,L
MAX_ROUNDS = 30
MAX_GENS = 200
POPULATION = 150
goal = -0.673667521146855 # http://benchmarkfcns.xyz/benchmarkfcns/kealefcn.html
delta = 0.0000001
CXPB = 0.45
MUTPB = 0.55
mute = False

# κρατάμε μόνο τον συνδυασμό που μας ενδιαφέρει
un_list=[]
bl_list=[0.5]
ga_list=[0.3]
sh_list=[]
se_list=[10]

strategy_round_testing(flag,goal,delta,MAX_ROUNDS,MAX_GENS,POPULATION,CXPB,MUTPB,un_list,bl_list,ga_list,sh_list,se_list,mute)

Πετύχαμε προσέγγιση μέχρι και το 6ο δεκαδικό ψηφίο, με επιτυχία σε κάθε γύρο για την εύρεση του ελαχίστου.

Όταν επιχειρήσαμε να κάνουμε προσέγγιση μέχρι και το 7ο δεκαδικό ψηφίο, το πλήθος των επιτυχιών έπεσε στις 26/30. Και πάλι το αποτέλεσμα κρίνεται αρκετά ικανοποιητικό.

Πετύχαμε επομένως την εύρεση ελαχίστου για τη μη-κλιμακούμενη συνάρτηση **`Keane`**, μέσα σε λογικά χρονικά πλαίσια, και με ικανοποιητική προσέγγιση ως προς τα δεκαδικά ψηφία.

---

---

---


## 2. Μελέτη κλιμακούμενης συνάρτησης
Η κλιμακούμενη συνάρτηση που θα μελετήσουμε είναι η **Stepint** :
\begin{equation}
f(\mathbf{x}) = 25 + \sum_{i=1}^{D}{\lfloor x_i \rfloor}, -5.12 \leq x_1 , \ldots , x_D \leq 5.12
\end{equation}
---

### Για D=2
- α) Εκτυπώστε ένα “3D” γράφημα της συνάρτησης $f(x1,x2)$ και περιγράψτε σύντομα τη μορφή της.

- β) Με την διαδικασία που ακολουθήσαμε προηγουμένως για τη μη κλιμακούμενη συνάρτηση, βρίσκουμε τον βέλτιστο γενετικό αλγόριθμο για το πρόβλημα. Προσαρμόζουμε τον κώδικα του πρώτου μέρους της άσκησης. Μπορεί να αντιγράφουμε ήδη έτοιμο κώδικα, αλλά το κάνουμε για να είναι ανεξάρτητες οι δύο ασκήσεις.

### Για D=1,10,20,50 και μεγαλύτερες διαστάσεις

Για τον βέλτιστο αλγόριθμο (που βρήκατε για 2 διαστάσεις) και για διαφορετικές τιμές/τάξεις μεγέθους D=1,10,20,50,... των διαστάσεων του πεδίου ορισμού (σταθερά MAX_GENS, MAX_ROUNDS>=10) τυπώστε πίνακα με: **αριθμό διαστάσεων**, **αριθμό επιτυχιών**, **μέσο ολικό ελάχιστο**, **μέσο αριθμό αποτιμήσεων** και **μέσο χρόνο** (προφανώς βασιστείτε στην αρχική βοηθητική συνάρτηση εκτύπωσης αποτελεσμάτων). Σημειώστε ότι οποιαδήποτε ποινή για άτομα εκτός ορισμού καθώς και τo DELTA προκειμένου να είναι αποτελεσματικά θα πρέπει εμπειρικά να οριστούν συναρτήσει του πλήθους των διαστάσεων.

- Σχολιάστε τα αποτελέσματα του πίνακα. Ποιες είναι οι αιτίες του φαινομένου; Πώς μπορούμε να βελτιώσουμε τα αποτελέσματα σε μεγάλες διαστάσεις για σταθερό αριθμό γενεών, τουλάχιστον ως προς το σκέλος της μέσης βέλτιστης τιμής (κυρίως) καθώς και των επιτυχιών;
- Κάντε μια δοκιμή (σε λογικά πλαίσια χρόνου) των αλλαγών που προτείνετε, τυπώστε εκ νέου τον πίνακα και σχολιάστε.

## Βιβλιογραφία

- Jamil, Momin, and Xin-She Yang. "A literature survey of benchmark functions for global optimisation problems." International Journal of Mathematical Modelling and Numerical Optimisation 4.2 (2013): 150-194. [link](https://goo.gl/qAhxNf)

- Whitley, Darrell, et al. "Evaluating evolutionary algorithms." Artificial intelligence 85.1-2 (1996): 245-276. [link](https://www.sciencedirect.com/science/article/pii/0004370295001247/pdf?md5=41088a944920336e5d8493160de27800&pid=1-s2.0-0004370295001247-main.pdf&_valck=1)

- "Gradient Free Optimization". Stanford AA222 - Introduction to Multidisciplinary Design Optimization handouts. [link](http://adl.stanford.edu/aa222/Lecture_Notes_files/chapter6_gradfree.pdf) 

- [DEAP Benchmarks functions](http://deap.readthedocs.io/en/master/api/benchmarks.html)

- [Global Optimization Test Functions Index](http://infinity77.net/global_optimization/test_functions.html)


## Σημείωσεις για την υλοποίηση

- Λόγω των πολλών επαναλήψεων είναι απαραίτητη και θα αξιολογηθεί η **εκτεταμένη χρήση συναρτήσεων**.
- Για απορίες και διευκρυνίσεις ως συνήθως συμβουλευτείτε πρώτα το [FAQ](https://goo.gl/igxHTY) και στη συνέχεια αν χρειαστεί μπορείτε να στείλετε mail στο [nnlab@islab.ntua.gr](mailto:nnlab@islab.ntua.gr)